In [1]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [5]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [2]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [6]:
data[1]['overall']

1.0

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [7]:
ratings = [ elt['overall'] for elt in data]

In [9]:
data[0]

{'reviewerID': 'A238V1XTSK9NFE',
 'asin': 'B00004VX3T',
 'reviewerName': 'Andrew Lynn',
 'helpful': [2, 2],
 'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatical

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [elt[self.key] for elt in X]
    


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
v = CustomTransformer('reviewText') 
x = v.fit_transform(data)
vectorizer = CountVectorizer(stop_words=STOP_WORDS)
vectorizer.fit_transform(x)


<80000x76675 sparse matrix of type '<class 'numpy.int64'>'
	with 3197884 stored elements in Compressed Sparse Row format>

In [12]:

bag_of_words_model = Pipeline([('gey_text', CustomTransformer('reviewText')),
                           ('vectorizer', CountVectorizer(stop_words=STOP_WORDS)),
                           ('ridge', GridSearchCV(Ridge(), param_grid={'alpha':np.logspace(-1,2,10)}, cv=5, verbose=1,n_jobs=1 ))
                              ])

In [13]:
import numpy as np
y = np.linspace(2,3,5)
pow(10, y)
np.logspace(-1,2,10)
pow(10, np.linspace(-1,2,10))

array([  0.1       ,   0.21544347,   0.46415888,   1.        ,
         2.15443469,   4.64158883,  10.        ,  21.5443469 ,
        46.41588834, 100.        ])

In [14]:
np.logspace?

In [15]:
cv = bag_of_words_model.named_steps['vectorizer']


In [16]:
bag_of_words_model.fit(data,ratings)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 14.1min finished


Pipeline(memory=None,
         steps=[('gey_text', CustomTransformer(key='reviewText')),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words={'a', 'about', 'above', 'across',
                                             'a...
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                             

In [17]:
bag_of_words_model.score(data,ratings)

0.4337251688968673

In [18]:
print(bag_of_words_model.named_steps['ridge'].best_params_)

{'alpha': 100.0}


In [19]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score:  1.0224665334624339


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [20]:

normalized_model =  Pipeline([('gey_text', CustomTransformer('reviewText')),
                           ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS)),
                           ('ridge', GridSearchCV(Ridge(), param_grid={'alpha':np.logspace(-1,2,10)}, cv=5, verbose=1,n_jobs=1 ))
                              ])

In [21]:
normalized_model.fit(data,ratings)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.4min finished


Pipeline(memory=None,
         steps=[('gey_text', CustomTransformer(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a'...
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                              tol=0.001),
 

In [22]:
normalized_model.score(data,ratings)

0.545231454341623

In [23]:
normalized_model.named_steps['ridge'].best_params_

{'alpha': 2.1544346900318834}

In [24]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score:  0.9976735015005194


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [25]:

bigrams_model = Pipeline([('gey_text', CustomTransformer('reviewText')),
                           ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(1, 2))),
                           ('ridge', GridSearchCV(Ridge(), param_grid={'alpha':np.logspace(-1,2,10)}, cv=5, verbose=1,n_jobs=1 ))
                              ])

In [ ]:
bigrams_model.fit(data,ratings)

In [ ]:
bigrams_model.score(data,ratings)

In [ ]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [2]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [8]:
del data, ratings

In [4]:
import numpy as np
import gzip
import ujson as json
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [ elt['overall'] for elt in data_polarity]


In [5]:
ratings = [ elt['overall'] for elt in  data_polarity]
ratings[0]

1.0

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import STOP_WORDS
polarity_model = Pipeline([('gey_text', CustomTransformer('reviewText')),
                   ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS )),
                    ('estimator',MultinomialNB())
                          ])

In [23]:
 polarity_model.fit(data_polarity,ratings)

Pipeline(memory=None,
         steps=[('gey_text', CustomTransformer(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a'...
                                             'also', 'although', 'always', 'am',
                                             'among', 'amongst', 'amount', 'an',
                                             'and', 'another', 'any', 'anyhow',
                                             'anyone', 'anything', 'anyway',
                                  

In [24]:
nb = polarity_model.named_steps['estimator']

array([[ -8.64595801,  -9.6369024 , -10.86439331, ..., -10.19829036,
        -10.71077731, -10.62308687],
       [ -9.35207212,  -9.30288304, -10.57273656, ..., -10.81783738,
        -10.81783738, -10.81783738]])

In [39]:
polarity = nb.feature_log_prob_[0,:] - nb.feature_log_prob_[1,:]

In [45]:
len(polarity)

25422

In [46]:
index = np.argsort(polarity)

In [63]:
extreme_index = np.hstack((index[:25],index[:-26:-1]) )

In [59]:
index_to_token = polarity_model.named_steps['vectorizer'].get_feature_names()

In [53]:
index_to_token[0]

'00'

In [64]:
top_50 = [ index_to_token[elt] for elt in extreme_index]

In [65]:
len(top_50)

50

In [66]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*